In [42]:
from os import path
from string import punctuation
from re import sub

from gensim.models import Word2Vec, Phrases, word2vec

translator = str.maketrans('', '', punctuation)

In [10]:
def remove_double_spaces(string):
    return sub(' +', ' ', string)

In [2]:
with open(path.join('data', 'pidgeons.txt'), 'r') as f:
    text = f.read()

In [30]:
def split_on_batches(text_sequence, batch_size=100):
    while text_sequence:
        yield list(find_neighbors(text_sequence[:batch_size]))
        text_sequence = text_sequence[batch_size:]

In [35]:
def find_neighbors(text_batch, window=3):
    words = remove_double_spaces(text_batch.translate(translator).replace('\n', ' ')).split(' ')
    for word_id, word in enumerate(words):
        lower_bound = word_id-window
        if word_id-window < 0:
            lower_bound = 0
        d = {}
        d[word] = words[lower_bound:word_id] + words[word_id+1:word_id+window]
        yield(d)

In [36]:
sents = list(split_on_batches(text, 100))[:30]

In [56]:
def split_on_batches(text_sequence, batch_size=100):
    while text_sequence:
        yield text_sequence[:batch_size]
        text_sequence = text_sequence[batch_size:]

sentences = [a.split(' ') for a in list(split_on_batches(text))]
#bigram_transformer = Phrases(sentences)
#bigram_sentences = bigram_transformer[sentences]
bigram_sentences = sentences

model=Word2Vec()
model.build_vocab(bigram_sentences)
model.save('wiki_vocab.model')

In [28]:
def split_on_batches(text_sequence, batch_size=100):
    while text_sequence:
        yield text_sequence[:batch_size]
        text_sequence = text_sequence[batch_size:]

model = Word2Vec() # sg and hs are the default parameters
model.build_vocab(text[:100].split(' '))
model.train(text[:100].split(' '), total_examples=100, epochs=3)


for i in list(split_on_batches(text)):
    model.build_vocab(i.split(' '), update=True)
    model.train(i.split(' '), total_examples=100, epochs=3)

In [57]:
model.wv.vocab

{'': <gensim.models.keyedvectors.Vocab at 0x11000ceb8>,
 '"I': <gensim.models.keyedvectors.Vocab at 0x10fcbdb38>,
 '"That': <gensim.models.keyedvectors.Vocab at 0x10fd1a0f0>,
 ',': <gensim.models.keyedvectors.Vocab at 0x10fcbd630>,
 '.': <gensim.models.keyedvectors.Vocab at 0x10fdbc9e8>,
 'A': <gensim.models.keyedvectors.Vocab at 0x1032bbe48>,
 'And': <gensim.models.keyedvectors.Vocab at 0x10fdbcb38>,
 'Black': <gensim.models.keyedvectors.Vocab at 0x10fd1a080>,
 'Blassenville': <gensim.models.keyedvectors.Vocab at 0x10fddae48>,
 'Branner': <gensim.models.keyedvectors.Vocab at 0x10fe85518>,
 "Branner's": <gensim.models.keyedvectors.Vocab at 0x10fcbd208>,
 'Buckner': <gensim.models.keyedvectors.Vocab at 0x10fd171d0>,
 "Buckner's": <gensim.models.keyedvectors.Vocab at 0x10fd17588>,
 'Buckner,': <gensim.models.keyedvectors.Vocab at 0x10fd17550>,
 'Buckner.': <gensim.models.keyedvectors.Vocab at 0x10fd17a58>,
 'But': <gensim.models.keyedvectors.Vocab at 0x10fdbc8d0>,
 'Celia': <gensim.model